Importation des packages

In [3]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO

Paramétrage de l'API

In [4]:
BASE_URL_API = https://apidf-preprod.cerema.fr""
#code_insee, nom =  "59512", "Roubaix" #"75015", "Paris"
#code_insee Lille = 59350
code_insee = "78220"
url = BASE_URL_API + f"/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350"
# %% 
print(url)

SyntaxError: invalid syntax (3147738439.py, line 1)

In [1]:
response = requests.get(url)#, params=params)
# Vérification du succès de la requête
if response.status_code == 200:
    data = response.json()  # On récupère les données au format JSON
else:
    print(f"Erreur : {response.status_code}")

NameError: name 'requests' is not defined

In [5]:
# Conversion en DataFrame pandas
df = pd.DataFrame(data["features"])

NameError: name 'data' is not defined

In [6]:
mut_gdf = gpd.read_file(StringIO(response.text))

NameError: name 'response' is not defined

In [114]:
mut_gdf.head()

,idmutinvar,idopendata,datemut,anneemut,coddep,libnatmut,vefa,valeurfonc,nbcomm,nbpar,nbparmut,sterr,nbvolmut,nblocmut,sbati,codtypbien,libtypbien,geometry
0,83a68c1f0c617ae2609e8ade54c11804,83a68c1f0c617ae2609e8ade54c11804,2020-09-10,2020,59,Vente,False,200000.00,1,1,0,0.00,0,1,37.00,121,UN APPARTEMENT,"MULTIPOLYGON (((3.05323 50.6372, 3.05317 50.63..."
1,45005cd8bafee591ed6debb57bbc6613,45005cd8bafee591ed6debb57bbc6613,2020-12-23,2020,59,Vente en l'état futur d'achèvement,True,238800.02,1,1,0,0.00,0,0,0.00,101,BATI - INDETERMINE : Vefa sans descriptif,"MULTIPOLYGON (((3.06685 50.62606, 3.06694 50.6..."
2,ebbcec3f97add3b68e7b452f8ba73f21,ebbcec3f97add3b68e7b452f8ba73f21,2020-12-17,2020,59,Vente,False,294000.00,1,1,0,0.00,0,1,40.00,121,UN APPARTEMENT,"MULTIPOLYGON (((3.05946 50.64373, 3.05957 50.6..."
3,7bcb49cdc14b9303042d3938d9ff7510,7bcb49cdc14b9303042d3938d9ff7510,2020-07-07,2020,59,Vente,False,1.00,1,1,0,0.00,0,1,54.00,121,UN APPARTEMENT,"MULTIPOLYGON (((3.05761 50.6304, 3.05759 50.63..."
4,aee36b8e79a346304326d5bbed32e4f5,aee36b8e79a346304326d5bbed32e4f5,2020-12-04,2020,59,Vente,False,4500.00,1,1,0,0.00,0,1,0.00,131,UNE DEPENDANCE,"MULTIPOLYGON (((3.0738 50.61993, 3.07384 50.61..."


Description des variables de ce dataframe :
- idmutinvar = identifiant unique entre les diff bases pour retrouver chaque mutation
- idopendata = identifiant de mutation utilisé depuis l'ouverture de données DVF
- datemut = date de signature de l'acte de vente
- anneemut = annee de signature du doc
- codedep
- libnatmut = Libellé de nature de mutation tel que défini dans la donnée source DVF 6 catégories : Vente, Vente en l'état futur d'achèvement, Adjudication, Echange, Vente terrain à bâtir et Expropriation.
- vefa = booléen selon si la vente est en VEFA (vente en l'état futur d'achèvement) ou non
- valeurfonc = valeur foncière en euro déclarée lors de la vente ; prix net vendeur, inclut la TVA mais n'inclut pas les frais de notaire, d'agence ou des meubles. 
- nbcomm = nombre de communes concernée par la mutation (i.e. sur lesquelles se trouve la parcelle cadastrale concernée par la mutation)
- nbpar = comme nbcomm mais pour les parcelles.
- nbparmut = nb de parcelles ayant muté (i.e. dont la pleine propriété a été transférée lors de la mutation)
- sterr = surface de terrain ayant muté (mutation = transfert de propriété, donc surface de terrain ayant muté = surface du terrain qui a été vendu)
- nbvolmut = variable obscure, flemme 
- nblocmut = nb de locaux ayant muté
- sbati = surface de l'ensemble du bati ayant muté
- codtypbien = typologie des biens ; on pourra s'en servir pour filter les maisons et appartements et virer tous les types de batiments bizarres (cf schéma ici) https://doc-datafoncier.cerema.fr/doc/dv3f/mutation/codtypbien
- libtypbien = comme codtypbien mais pour la base DV3F et non issu de juste DVF
- geometry

Il va falloir nettoyer notre jeu de toutes les observations qui ne nous intéressent pas. Par exemple, pour enlever toutes les transactions qui ne sont pas des ventes :

In [115]:
mut_gdf= mut_gdf[mut_gdf["libnatmut"] == "Vente"]
len(mut_gdf)

86

Noter qu'on peut donc travailler sur des colonnes de types object sans changer ce type s'il s'agit de texte (on n'a pas besoin de les transformer en string). On est sûr que le filtre a fonctionné car on a moins d'observation après filtre.

Mais on a un problème avec la surface transférée : parfois on voit 0 pour le terrain vendu ET pour le bâti vendu (respectivement sterr et sbati). Il faut écarter ces anomalies. Mais ces deux variables sont du type "object", ce qui nous empêche de les comparer numériquement.

In [116]:
mut_gdf.dtypes

idmutinvar            object
idopendata            object
datemut       datetime64[ms]
anneemut               int32
coddep                object
libnatmut             object
vefa                    bool
valeurfonc            object
nbcomm                 int32
nbpar                  int32
nbparmut               int32
sterr                 object
nbvolmut               int32
nblocmut               int32
sbati                 object
codtypbien            object
libtypbien            object
geometry            geometry
dtype: object

On va donc changer le type de ces deux variables en décimaux.

In [117]:
mut_gdf['sterr']  = mut_gdf["sterr"].astype(float)
mut_gdf['sbati']  = mut_gdf["sbati"].astype(float)

In [118]:
mut_gdf.dtypes

idmutinvar            object
idopendata            object
datemut       datetime64[ms]
anneemut               int32
coddep                object
libnatmut             object
vefa                    bool
valeurfonc            object
nbcomm                 int32
nbpar                  int32
nbparmut               int32
sterr                float64
nbvolmut               int32
nblocmut               int32
sbati                float64
codtypbien            object
libtypbien            object
geometry            geometry
dtype: object

Et maintenant on peut appliquer un filtre : (peut-être d'une meilleure manière que d'enchaîner les filtres).

In [119]:
mut_gdf["stotale"] = mut_gdf["sterr"] + mut_gdf["sbati"]
mut_gdf = mut_gdf[mut_gdf["stotale"] != 0]
len(mut_gdf)

81

Voilà le genre de filtre qu'on veut appliquer sur notre jeu de données. 

On peut chercher parmi nos observations le maximum et le minimum pour une variable. Par exemple, pour le prix de l'achat ("valeurfonc").

In [120]:
mut_gdf['valeurfonc']  = mut_gdf["valeurfonc"].astype(float)


In [121]:
maxi = 0
mini = 10000000000000000000000000000000000000
for index, row in mut_gdf.iterrows():
    if row["valeurfonc"] > maxi : 
        maxi = row["valeurfonc"]
    if row["valeurfonc"] < mini : 
        mini = row["valeurfonc"]
print(mini, maxi)

1.0 957000.0


Ce qui laisse bien l'impression d'avoir des outliers.